In [1]:
import os
import pandas as pd
import networkx as nx
import numpy as np
data_dir = 'real_world_data'
data_fn = 'COVIDW1_W6.csv'
data_path = os.path.join(data_dir, data_fn)

In [2]:
data = pd.read_csv(data_path)

/tmp/ipykernel_16312/1675668717.py:1: DtypeWarning: Columns (0,5,1462,1463,1464,1465,1466,1467,1469,1470,1471,1472,1474,1475,1476,1477,1478,1479,1481,1490,1491,1505,1506,1507,1508,1509,1510,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1523,1524,1525,1526,1527,1528,1529,1530,1536,1537,1538,1539,1540,1541,1542,1543,1544,1545,1546,1547,1548,1549,1550,1551,1552,1553,1554,1555,1556,1557,1558,1559,1560,1561,1562,1564,1565,1566,1567,1568,1569,1570,1571,1572,1573,1574,1575,1576,1577,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1603,1616,1617,1618,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1630,1631,1632,1633,1634,1635,1636,1637,1638,1639,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682,1683,1684,1685,1686,1687,1688,1689,1690,1691,

In [3]:
w2_cols = []
for col in data.columns:
    if 'W2' in col:
        w2_cols.append(col)
    if 'W2_Relationships' in col:
        print(col)

W2_Relationships1
W2_Relationships2
W2_Relationships3
W2_Relationships4


In [4]:

    
    
w2_age = ['W2_Age_year']
w2_keyworker = ['W2_Keyworker']
w2_adults = ['W2_Adults_household']
w2_children = ['W2_Children_household'] 
w2_income = ['W2_Change_Income'] 
w2_covid_anx = ['W2_COVID19_anxiety']
w2_risk1month = ['W2_RISK_1month'] 

w2_gender = ['W2_Gender']
w2_attachment = ['W2_Attach_Style']
w2_loneliness = ['W2_Loneliness1',
                'W2_Loneliness2',
                'W2_Loneliness3']
w2_chronicill = ['W2_Chronic_illness_self']
w2_rel_stat = ['W2_Relationship']
w2_social_dist = ['W2_SocialDistance1',
                'W2_SocialDistance2',
                'W2_SocialDistance3',
                'W2_SocialDistance4',
                'W2_SocialDistance5',
                'W2_SocialDistance6',
                'W2_SocialDistance7',
                'W2_SocialDistance8',
                'W2_SocialDistance9',
                'W2_SocialDistance10',
                'W2_SocialDistance11',
                'W2_SocialDistance12',
                'W2_SocialDistance13',
                'W2_SocialDistance14',
                'W2_SocialDistance15',
                'W2_SocialDistance16']

w2_depression = ['W2_Dep_1',
            'W2_Dep_2',
            'W2_Dep_3',
            'W2_Dep_4',
            'W2_Dep_5',
            'W2_Dep_6',
            'W2_Dep_7',
            'W2_Dep_8',
            'W2_Dep_9']
w2_anx = ['W2_GAD_1',
        'W2_GAD_2',
        'W2_GAD_3',
        'W2_GAD_4',
        'W2_GAD_5',
        'W2_GAD_6',
        'W2_GAD_7']

# wave 3 variables
w3_age = ['W2_Age_year']  
w3_gender = ['W3_Age_year']
w3_attachment = ['W3_Attach_style']
# wave 1 variables
w1_age = ['W1_Age_year']  
w1_gender = ['W1_Age_year']
w1_attachment = ['W3_Attach_style']  # no W1 value exists, so just try W3 again

all_vars = [w2_anx, w2_depression, w2_social_dist, w2_age, w2_keyworker, w2_adults, w2_children, w2_income, w2_covid_anx, w2_risk1month,
            w2_gender, w2_attachment, w2_loneliness, w2_chronicill, w2_rel_stat]

In [5]:
# convert to float:
def convert_columns_to_float(df, columns):
    df[columns] = df[columns].apply(pd.to_numeric, errors='coerce')
    
    
for construct in [item for sublist in all_vars for item in sublist]:
    convert_columns_to_float(data, construct)


In [6]:

# forward backward filling for sets to fill:
sets_to_fill = [w2_age, w2_gender, w2_attachment]
for col_w2, col_w1, col_w3 in zip(sets_to_fill, [w1_age, w1_gender, w1_attachment], [w3_age, w3_gender, w3_attachment]):
    
    data[col_w2[0]] = data[col_w2[0]].combine_first(data[col_w1[0]])  # Fill missing with wave 1 values
    data[col_w2[0]] = data[col_w2[0]].combine_first(data[col_w3[0]]) 


In [7]:
# list wise deletion for relevant variables
data = data[[item for sublist in all_vars for item in sublist]]
print(data.shape)
data = data.dropna()
print(data.shape)

(5364, 46)
(1406, 46)


In [9]:
# deal with -99 and other anomolous values:

# remove -99 in W2_Keyworker
# change -9 to 0 in keyworker
# change -9 to 0 in Adults household
# remove -99 in children household
# change -9 to -1 in children household
# remove -99 from attachment style


# Step 1: Listwise delete rows where -99 is the value of W2 Keyworker
data = data[data[w2_keyworker[0]] != -99]

# Step 2: Change -9 to 0.0 in W2 Keyworker
data[w2_keyworker[0]] = data[w2_keyworker[0]].replace(-9, 0.0)

# Step 3: Change -9 to 0.0 in Adults household
data[w2_adults[0]] = data[w2_adults[0]].replace(-9, 0.0)

# Step 4: Listwise delete rows where -99 is the value of Children household
data = data[data[w2_children[0]] != -99]

# Step 5: Change -9 to -1.0 in Children household
data[w2_children[0]] = data[w2_children[0]].replace(-9, -1.0)

# Step 6: Listwise delete rows where -99 is the value of Attachment style
data = data[data[w2_attachment[0]] != -99]

print(data.shape)

(895, 46)


In [10]:
# export
data.to_csv(os.path.join(data_dir, 'real_world_dataset.csv'), index=False)


